# CVBS Deep Memory Generation Playback

Plays back captured CVBS data using Deep Memory Generation (DMG).

## Workflow
1. **On local machine:** Convert 8-bit capture to float32:
   ```bash
   python convert_to_playback.py capture.bin
   scp capture.f32 root@192.168.0.6:/home/jupyter/cvbs_project/cvbs_captures/
   ```

2. **On Red Pitaya:** Run this notebook to play back

## Prerequisites
- Red Pitaya OS 2.07-48 or later
- DMA region expanded to 128 MB
- Float32 capture file in /home/jupyter/cvbs_project/cvbs_captures/

## Configuration

In [ ]:
# === CONFIGURATION ===

# Path to float32 waveform file (created by convert_to_playback.py)
WAVEFORM_FILE = "/home/jupyter/cvbs_project/cvbs_captures/cvbs_playback.f32"

# Output settings
DECIMATION = 8          # 125 MS/s / 8 = 15.625 MS/s
SAMPLE_RATE = 125e6 / DECIMATION

# Loop count: 0 = play once, N = loop N additional times, -1 = infinite
LOOP_COUNT = 0

# Output channel (1 or 2)
OUTPUT_CHANNEL = 1

import os
if os.path.exists(WAVEFORM_FILE):
    size_mb = os.path.getsize(WAVEFORM_FILE) / 1024 / 1024
    samples = os.path.getsize(WAVEFORM_FILE) // 4
    duration = samples / SAMPLE_RATE
    print(f"Waveform: {WAVEFORM_FILE}")
    print(f"Size: {size_mb:.2f} MB ({samples:,} samples)")
    print(f"Duration: {duration:.3f}s @ {SAMPLE_RATE/1e6:.3f} MS/s")
else:
    print(f"ERROR: File not found: {WAVEFORM_FILE}")

## Initialize and Load

In [ ]:
import sys
import time
import gc
import numpy as np

sys.path.insert(0, "/opt/redpitaya/lib/python")
from rp_overlay import overlay
import rp

# Initialize
fpga = overlay()
rp.rp_Init()

CH = rp.RP_CH_1 if OUTPUT_CHANNEL == 1 else rp.RP_CH_2
print(f"Red Pitaya initialized, output: RF OUT {OUTPUT_CHANNEL}")

# Check DMG memory
mem_result = rp.rp_GenAxiGetMemoryRegion()
print(f"rp_GenAxiGetMemoryRegion() returned: {mem_result}")
MEM_START = mem_result[1]
MEM_SIZE = mem_result[2]
print(f"DMG Memory: start=0x{MEM_START:08X}, size={MEM_SIZE:,} bytes ({MEM_SIZE/1024/1024:.1f} MB)")

# List available DMG-related functions for reference
dmg_funcs = [f for f in dir(rp) if 'Axi' in f or 'axi' in f]
print(f"\nAvailable Axi/DMG functions ({len(dmg_funcs)}):")
for f in sorted(dmg_funcs):
    print(f"  {f}")

In [ ]:
# Load waveform
print(f"Loading {WAVEFORM_FILE}...")
waveform = np.fromfile(WAVEFORM_FILE, dtype=np.float32)

# Truncate if exceeds DMG memory (remember: int16 storage = 2 bytes/sample)
max_samples = MEM_SIZE // 2  # MEM_SIZE is bytes, int16 = 2 bytes
if len(waveform) > max_samples:
    print(f"Truncating {len(waveform):,} to {max_samples:,} samples (DMG limit)")
    waveform = waveform[:max_samples]

# Truncate to 128-byte alignment (required by API)
# 128 bytes = 64 int16 samples
ALIGN_BYTES = 128
SAMPLES_PER_ALIGN = ALIGN_BYTES // 2  # 64 samples
original_len = len(waveform)
remainder = original_len % SAMPLES_PER_ALIGN

if remainder != 0:
    aligned_len = original_len - remainder
    print(f"128-byte aligning: {original_len:,} -> {aligned_len:,} samples (truncated {remainder})")
    waveform = waveform[:aligned_len]
else:
    print(f"Already 128-byte aligned: {original_len:,} samples")

DURATION = len(waveform) / SAMPLE_RATE
print(f"Loaded: {len(waveform):,} samples ({DURATION:.3f}s)")
print(f"Range: {waveform.min():.3f} to {waveform.max():.3f}")

## Write to DMG

In [ ]:
# Configure DMG
rp.rp_GenReset()

# Calculate memory requirements
# NOTE: DMG stores data as int16 internally (2 bytes/sample), not float32
# The API converts float -> int16 when writing
waveform_bytes = len(waveform) * 2  # int16 = 2 bytes per sample
mem_end = MEM_START + waveform_bytes

print(f"=== DMG Memory Debug ===")
print(f"MEM_START: 0x{MEM_START:08X} ({MEM_START})")
print(f"MEM_SIZE:  {MEM_SIZE} bytes ({MEM_SIZE/1024/1024:.2f} MB)")
print(f"Waveform:  {len(waveform):,} samples = {waveform_bytes:,} bytes ({waveform_bytes/1024/1024:.2f} MB as int16)")
print(f"mem_end:   0x{mem_end:08X} ({mem_end})")
print(f"Requested: {mem_end - MEM_START:,} bytes")

# Check 128-byte alignment (required by API)
if waveform_bytes % 128 != 0:
    print(f"WARNING: Not 128-byte aligned (remainder: {waveform_bytes % 128})")

# Reserve memory
print(f"\nReserving memory...")
reserve_result = rp.rp_GenAxiReserveMemory(CH, MEM_START, mem_end)
print(f"rp_GenAxiReserveMemory returned: {reserve_result}")

# Set decimation
dec_result = rp.rp_GenAxiSetDecimationFactor(CH, DECIMATION)
print(f"rp_GenAxiSetDecimationFactor({DECIMATION}) returned: {dec_result}")

# Write waveform (API converts float32 -> int16 internally)
print(f"\nWriting {len(waveform):,} float samples to DMG...")
t0 = time.time()
result = rp.rp_GenAxiWriteWaveform(CH, waveform)
t1 = time.time()

print(f"rp_GenAxiWriteWaveform returned: {result}")
if result == 0:
    print(f"Write complete: {t1-t0:.2f}s ({waveform_bytes/(t1-t0)/1024/1024:.1f} MB/s)")
else:
    print(f"WARNING: Write returned non-zero: {result}")

# Free waveform from Python memory (it's now in DMG)
del waveform
gc.collect()
print("\nWaveform loaded to DMG, Python memory freed")

## Playback Controls

In [ ]:
def play(loops=0):
    """Play the waveform. loops: 0=once, N=N+1 times, -1=infinite"""
    rp.rp_GenOutEnable(CH)
    rp.rp_GenAxiSetEnable(CH, True)
    
    if loops < 0:
        print(f"Playing infinitely ({DURATION:.3f}s per loop)")
        print("Call stop() to stop")
        return
    
    total = loops + 1
    print(f"Playing {total}x ({DURATION*total:.2f}s total)...")
    
    for i in range(total):
        if i > 0:
            rp.rp_GenAxiSetEnable(CH, True)
        time.sleep(DURATION)
    
    stop()
    print("Done")

def stop():
    """Stop playback"""
    rp.rp_GenAxiSetEnable(CH, False)
    rp.rp_GenOutDisable(CH)
    print("Stopped")

print("Ready! Use:")
print(f"  play()      - Play once ({DURATION:.3f}s)")
print(f"  play(5)     - Play 6 times")
print(f"  play(-1)    - Play infinitely")
print(f"  stop()      - Stop playback")

In [ ]:
# Play with configured loop count
play(LOOP_COUNT)

In [ ]:
# Stop playback
stop()

## Cleanup

In [ ]:
stop()
rp.rp_GenAxiReleaseMemory(CH)
rp.rp_GenReset()
rp.rp_Release()
print("Resources released")